In [17]:
pip install confluent_kafka

SyntaxError: invalid syntax (900263671.py, line 2)

In [2]:
pip install ess_streaming_data_types

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import datetime
import logging
from streaming_data_types import serialise_wrdn
import ipywidgets as widgets
from IPython.display import display
import pprint
import json
import functools

import sfi_generator as sfig

In [4]:
config = sfig.load_config('sfi_generator_config_docker_local.json')

In [5]:
output_results = widgets.Output(
    layout={"border":"1px solid gray","width":"auto"}
)

In [6]:
ta_message_preview = widgets.Textarea(
    value='',
    placeholder='Done Writing message in json format',
    description='',
    disabled=False,
    layout={
        'width': 'auto',
        'height': '400px'
    }
)

In [7]:
l_message_preview = widgets.Label(
    value="Message Preview"
)

In [8]:
b_prep_message = widgets.Button(
    description='Prepare message',
    disabled=False,
    button_style='',
    tooltip='Create next message to send and preview it',
    icon='eye'
)

In [9]:
def prep_message_click(b,text_area=None,config=None):
    dict_message = sfig.prep_message(config)
    formatted_message = json.dumps(dict_message, indent=4)
    text_area.value = formatted_message
    output_results.clear_output()

In [10]:
b_prep_message.on_click(
    functools.partial(
        prep_message_click,
        text_area=ta_message_preview,
        config=config
    )
)

In [11]:
b_send_message = widgets.Button(
    description='Send message',
    disabled=False,
    button_style='',
    tooltip='Send message in preview',
    icon='paper-plane'
)

In [12]:
def collect_output(message):
    output_results.append_stdout(message)

In [13]:
def send_message_click(b,text_area=None,config=None):
    dict_message = json.loads(text_area.value)
    serialized_message = sfig.serialize_message(dict_message)
    sfig.post_message(config,serialized_message,log_message=collect_output)

In [14]:
b_send_message.on_click(
    functools.partial(
        send_message_click,
        text_area=ta_message_preview,
        config=config
    )
)

In [15]:
layout = widgets.VBox(
    [
        b_prep_message,
        l_message_preview,
        ta_message_preview,
        b_send_message,
        output_results
    ],
    layout={
        'weight': 'auto',
        'height': 'auto'
    }
)

In [16]:
display(layout)